In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Viet_AI/Data')
print(os.listdir("."))

['train.csv', 'MLE_at_Epic_Software-SOLUTIONS.ipynb', 'U-net_TF2_retinal_image_segmentation_STARE.ipynb', 'U-net_binary_DB1.ipynb', 'resnet50.h5', 'my_model.h5', 'VietAI.hdf5', 'VietAI_Basline.ipynb', 'test', '.ipynb_checkpoints', 'train', 'logs', 'VietAI_multi-label.ipynb', 'mclass_data.npz', 'CNN_multi.h5', 'data.npz', 'VietAI_Tansfer_learning (1).ipynb', 'TF_30.h5', 'TF_conv_30.h5', 'VietAI_Tansfer_learning-conv.ipynb', 'LP_TF.npz', 'LP_TF_test.npz', 'TF.h5', 'VietAI_Tansfer_learning.ipynb', 'VietAI-visualize.ipynb', 'VietAI-visualize-TF.ipynb', 'VietAI_multi_label-multi_class.ipynb', 'ipykernel_launcher.py_plot.png', 'Viet_AI_LP_TF_noFalse_all_data.ipynb', 'TF_half_30.h5', 'LP_TF_half.npz', 'LP_TF_half_test.npz', 'VietAI-Label_propagate_TF_data_create.ipynb', 'Viet_AI_Results.gsheet']


In [ ]:
## Importing required Libraries
import os
import tensorflow as tf
from tensorboard.plugins import projector
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import accuracy_score, fbeta_score

In [ ]:
#Now load the model_data
# Load the npy files
npzfile=np.load('LP_TF_half.npz')
print(npzfile.files)
X=np.squeeze(npzfile['arr_0'])
Y=npzfile['arr_1']
UX=np.squeeze(npzfile['arr_2'])
UY=npzfile['arr_3']


['arr_0', 'arr_1', 'arr_2', 'arr_3']


In [ ]:
print(np.shape(X), np.shape(Y))

(2748, 512) (2748,)


In [ ]:
vals,count=np.unique(Y, return_counts=True)
print(vals, count)

[  1   2   4   6   8  10  16  18  20  24  26  32  34  36  40  42  48  56
  64  66  68  70  72  74  80  82  84  90  96  98 100 104 106 120] [425 141 239   1  25  59 253  23  25   3  13 166   4   1 120   2  14   3
 540  42 171   2   1  53 123   8  13   7 143   2   4 119   2   1]


In [ ]:
npzfile1=np.load('LP_TF_half_test.npz')
NUX=np.squeeze(npzfile1['arr_0'])
print(np.shape(NUX))

(350, 512)


In [ ]:
#Now load the model_data
# Load the npy files
npzfile=np.load('data.npz')
print(npzfile.files)
trainX=np.squeeze(npzfile['arr_0'])
trainY=npzfile['arr_1']
testX=np.squeeze(npzfile['arr_2'])
testY=npzfile['arr_3']


['arr_0', 'arr_1', 'arr_2', 'arr_3']


In [ ]:
np.shape(X), np.shape(trainX)

((2748, 512), (2748, 256, 256, 3))

In [ ]:
# Next load the test folder images
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from skimage import exposure

def load_dataset(img_path):
    images= list()
    # enumerate files in the directory
    files=os.listdir(img_path)
    l=len(files)
    for idx in range(l):
    # load image
        #print(idx)
        filename=files[idx]
        img = load_img(img_path + filename, target_size=(256,256), color_mode='rgb')
        img = img_to_array(img, dtype='uint8')
        # convert to numpy array
         #Contrast stretching
        p2, p98 = np.percentile(img, (2, 98))
        img = exposure.rescale_intensity(img, in_range=(p2, p98))
        
        images.append(img)
        
    X = np.asarray(images, dtype='uint8')
    return X

In [ ]:
NLX=load_dataset('./test/test/')

# Now we have [X, Y, UX, UY, NLX]->[trainX, trainY, testX, testY] as [TX, TY, TTX, TTY]

In [ ]:
#Reduce dimensions of X to 10 to speed up process
pca = PCA(n_components=35)
pca.fit(X)
X1=pca.transform(X)
UX1=pca.transform(UX)
NUX1=pca.transform(NUX)

print(np.shape(X1))

(2748, 35)


In [ ]:
#Normalize [X1, UX1, NUX1]
X1_n = (X1-X1.min(axis=0)) / (X1.max(axis=0)-X1.min(axis=0))
X1=X1_n

UX1_n = (UX1-UX1.min(axis=0)) / (UX1.max(axis=0)-UX1.min(axis=0))
UX1=UX1_n

NUX1_n = (NUX1-NUX1.min(axis=0)) / (NUX1.max(axis=0)-NUX1.min(axis=0))
NUX1=NUX1_n
# Next, we can visualize the clusters

In [ ]:
YO=-1*np.ones((np.shape(UY)[0]+np.shape(NUX)[0]))
print(np.shape(YO))
1037-350

(1037,)


687

In [ ]:
def return_metrics(y_true,y_pred):
  #y_true is binary [0,1], so is y_pred
  #return precision, recall, IOU, F1, Fbeta
  tp = np.sum(y_true * y_pred)
  fp = np.sum((y_pred - y_true)>0)
  fn = np.sum((y_true - y_pred)>0)
  #print(tp)
  eps=0.000001
	# calculate precision
  p = tp / (tp + fp + eps)
	# calculate recall
  r = tp / (tp + fn + eps)
  beta=2*2
  Fbeta=np.mean((1 + beta) * (p * r) / (beta * p + r + eps))
  return p, r, Fbeta

In [ ]:
def decimal_to_bin(num):
  res = [int(i) for i in bin(num)[2:]]
  result=np.zeros((7))
  l=(len(res))
  if l<7:
    result[-l:]=res
  else:
    result=res
  return (result)

In [ ]:
decimal_to_bin(108)

[1, 1, 0, 1, 1, 0, 0]

In [ ]:
# Convert UY labels
true_labels=np.zeros((np.shape(UY)[0],7))
for i in range(np.shape(UY)[0]):
  true_labels[i]=decimal_to_bin(UY[i])

In [ ]:
#Step 4.1: LP averaged across all 7 classes (equally likely)
import matplotlib.pyplot as plt
# select a random sample without replacement
from random import seed
from random import sample
# seed random number generator
drop=0.5#Fraction of population to be randomly dropped
n=np.shape(X)[0]
print(n)
sequence=range(0,n)
########################################################################
Xn=np.copy(X)
Ynd=np.copy(Y)
ns=np.round(drop*n).astype(int)
print('num of samples dropped=',ns)
dropI= sample(sequence, ns)
Ydrop=Ynd[dropI]
Ynd[dropI]=-1
no_drop=sequence
no_drop=np.delete(no_drop,dropI, 0)
print(np.shape(no_drop))
########################################################################
#Keep record of training images not dropped (50% training). To add glaucoma, ME and RVO to it.
TX=trainX[no_drop]
TY=trainY[no_drop]





2748
num of samples dropped= 1374
(1374,)


In [ ]:
#Now combine (Xn, UX1, NUX1)
Xn=np.concatenate((X1,UX1,NUX1))
Ynd=np.concatenate((Ynd,YO))
dropI=np.concatenate((dropI,range(np.shape(X)[0],np.shape(Xn)[0])))
#Thus (Xn,Ynd) is the new data that will be subjected to label propagation
        ################################### so random label removal works!
#Step 4.2: Apply Label propagation to [X,UX,NUX]
lp_model = LabelSpreading(kernel='rbf', gamma=20, alpha=0.2) #high alpha means labels an change till stable
            # Train the model
        #np.savetxt('test.out', Ynd, delimiter=',') 
lp_model.fit(Xn, Ynd)


        #Step 4.3: Predict Model label for the dropped samples
pred = lp_model.transduction_[dropI]
pred=pred.astype(int)
print(np.shape(pred))
        #print('Predictions=',pred)#pred[np.shape(X)[0]:np.shape(X)[0]+np.shape(UX)[0]])
        #print('Groundtruth=',UY)
        ############Predictions to be measured#######################
predictions=pred[ns:(ns+np.shape(UY)[0])] #This is the testY images
lab_preds_train=pred[0:ns] # This is the trainY
lab_preds_test=pred[(ns+np.shape(UY)[0]):(ns+np.shape(YO)[0])] #This is the unlabelled test data
print(lab_preds_test.shape)

(2411,)
(350,)


In [ ]:
np.shape(trainX), np.shape(trainY)

((2748, 256, 256, 3), (2748, 7))

In [ ]:
#Lets find Glaucoma (16), ME (8), RVO (2) from predictions
idx_train_G=np.where(Ydrop==16)
idx_test_G=np.where(lab_preds_test==16)
idx_train_ME=np.where(Ydrop==8)
idx_test_ME=np.where(lab_preds_test==8)
idx_train_RVO=np.where(Ydrop==2)
idx_test_RVO=np.where(lab_preds_test==2)
print(np.shape(idx_train_G),np.shape(idx_test_G), np.shape(idx_train_ME),np.shape(idx_test_ME), np.shape(idx_train_RVO), np.shape(idx_test_RVO))

(1, 122) (1, 22) (1, 15) (1, 0) (1, 69) (1, 5)


In [ ]:
#Add samples with Glaucoma, ME and RVO from training data
TY=np.concatenate((TY,(trainY[dropI[idx_train_G]])), axis=0)
print(np.shape(TY))

(1496, 7)


In [ ]:
TX=np.concatenate((TX,(trainX[dropI[idx_train_G]])), axis=0)
print(np.shape(TX))

(1496, 256, 256, 3)


In [ ]:
#Repeat for ME and RVO
TY=np.concatenate((TY,(trainY[dropI[idx_train_ME]])), axis=0)
TX=np.concatenate((TX,(trainX[dropI[idx_train_ME]])), axis=0)
print(np.shape(TY), np.shape(TX))

(1511, 7) (1511, 256, 256, 3)


In [ ]:
TY=np.concatenate((TY,(trainY[dropI[idx_train_RVO]])), axis=0)
TX=np.concatenate((TX,(trainX[dropI[idx_train_RVO]])), axis=0)
print(np.shape(TY), np.shape(TX))

(1580, 7) (1580, 256, 256, 3)


In [ ]:
#Finally append NLX and its labels to trainX
NLY=np.zeros((350,7))
for i in range(np.shape(NLX)[0]):
  NLY[i]=decimal_to_bin(lab_preds_test[i])

In [ ]:
TX=np.concatenate((TX,NLX),axis=0)
TY=np.concatenate((TY,NLY),axis=0)
print(np.shape(TY), np.shape(TX))

(1930, 7) (1930, 256, 256, 3)


In [ ]:
np.savez('new_train_data', TX, TY, testX, testY)